This experiment is aimed at confirm the hypothesis that 
we can achieve high scores in all human sim dataset.

noun_ng, noun_sim, noun_ws353 and so on

To make sure the design of 

In [23]:
import numpy as np
from sklearn.linear_model import LinearRegression
import spacy
from sematch.evaluation import WordSimDataset
import torch
from embed import ranking, get_adjacency_by_idx, eval_human
from nltk.corpus import wordnet
import eval_scws
from model import PoincareDistance
import time
import torch as th
from data import slurp
import os
from IPython.display import display
from model import SNEmbedding
import evaluation
import json
import re
import pickle
from collections import defaultdict
import pandas as pd
import matplotlib.pyplot as plt
from collections import defaultdict
import seaborn as sns

word_sim_data = WordSimDataset()
refer_pairs, refer_human_ground_truth = word_sim_data.load_dataset('noun_rg')
refer_pairs = list(refer_pairs)
datasets = ['noun_rg', 'noun_mc', 'noun_ws353', 'noun_ws353-sim', 'noun_simlex']
ground_truth_pairs = [word_sim_data.load_dataset(dataset) for dataset in datasets]
ground_truth_pairs = [(list(p), s) for p, s in ground_truth_pairs]



In [39]:
ans = defaultdict(lambda: {})
conflicts = defaultdict(lambda: defaultdict(lambda: []))
n_conflicts = 0
n_all = 0
for pairs, s in ground_truth_pairs:
    for (a, b), sim in zip(pairs, s):
        conflicts[b][a].append(sim)
        conflicts[a][b].append(sim)
        n_conflicts += 1
        n_all += 1
            
print(n_conflicts, '/',n_all)
print("So there are lots of conflicts in the dataset")
print(f'The size of all conflict pairs = {len(conflicts) // 2}')


1310 / 1310
So there are lots of conflicts in the dataset
The size of all conflict pairs = 534


In [37]:
ga = []
gb = []
for a, items in conflicts.items():
    for b, sim in items.items():
        scores = sim + [ans[a][b]]
        ga.append(min(scores))
        gb.append(max(scores))
print(np.corrcoef(ga, gb)[0, 1])

0.8150662786741674


In [44]:
for name, (pairs, human_sims) in zip(datasets, ground_truth_pairs):
    pred = []
    for (a, b), sim in zip(pairs, human_sims):
        scores = conflicts[a][b]
        if max(scores) - sim > sim - min(scores):
            pred.append(max(scores))
        else:
            pred.append(min(scores))
    print(name, np.corrcoef(pred, human_sims)[0, 1])

noun_rg 0.7677787216587257
noun_mc 0.9499313508769889
noun_ws353 0.8572644516070094
noun_ws353-sim 0.834030842374988
noun_simlex 0.9856734576207407


所以这几个数据集不会直接冲突。

这部分不加到论文里面，但是答辩如果问起来可以看一下。